In [1]:
# --- NOTEBOOK 06: THE BUREAU PLANET ---

import pandas as pd
import numpy as np
import gc
import logging

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Config:
    # We need the raw CSVs for the external tables
    # Adjust path if your files are in a specific folder (e.g., 'input/')
    BUREAU_PATH = 'data/bureau.csv'
    BUREAU_BAL_PATH = 'data/bureau_balance.csv'
    
    # We load our engineered data from NB 05 to join onto
    MAIN_DATA_PATH = 'engineered_data.parquet'
    
    OUTPUT_PATH = 'train_bureau_merged.parquet'

logger.info("Notebook 06 Initialized.")

2026-01-20 09:37:40,132 - INFO - Notebook 06 Initialized.


In [2]:
def aggregate_bureau_balance():
    logger.info("Loading Bureau Balance...")
    bb = pd.read_csv(Config.BUREAU_BAL_PATH)
    
    # One-Hot Encode the Status (0=Paid, 1=Late, C=Closed, X=Unknown)
    # We want to know: "How many months was the user in Status '5' (Very Late)?"
    bb_cats = pd.get_dummies(bb['STATUS'], prefix='B_BAL_STATUS')
    
    # Combine with ID
    bb = pd.concat([bb[['SK_ID_BUREAU']], bb_cats], axis=1)
    
    # Aggregations per LOAN (SK_ID_BUREAU)
    # We calculate the Mean (percentage of time in that status)
    # and Sum (total months in that status)
    agg_dict = {col: ['mean', 'sum'] for col in bb_cats.columns}
    
    logger.info("Aggregating Balance by Loan ID...")
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(agg_dict)
    
    # Flatten MultiIndex Columns (e.g., ('B_BAL_STATUS_0', 'mean') -> 'B_BAL_STATUS_0_MEAN')
    bb_agg.columns = [f"{c[0]}_{c[1].upper()}" for c in bb_agg.columns]
    
    # Add a count (Length of history for that loan)
    bb_count = bb.groupby('SK_ID_BUREAU').size().rename('B_BAL_MONTHS_HISTORY')
    bb_agg = bb_agg.join(bb_count)
    
    logger.info(f"Balance Aggregated. Shape: {bb_agg.shape}")
    return bb_agg

# Execute
bb_agg = aggregate_bureau_balance()
# Free memory
gc.collect()

2026-01-20 09:37:55,541 - INFO - Loading Bureau Balance...
2026-01-20 09:38:02,670 - INFO - Aggregating Balance by Loan ID...
2026-01-20 09:38:09,582 - INFO - Balance Aggregated. Shape: (817395, 17)


29

In [3]:
def aggregate_bureau(bb_agg):
    logger.info("Loading Bureau Data...")
    bureau = pd.read_csv(Config.BUREAU_PATH)
    
    # 1. JOIN BALANCE STATS
    # Connect the monthly history to the loan
    bureau = bureau.join(bb_agg, on='SK_ID_BUREAU', how='left')
    
    # Drop the Loan ID (we are rolling up to User ID now)
    bureau.drop(columns=['SK_ID_BUREAU'], inplace=True)
    
    # 2. DEFINE AGGREGATIONS
    # For Numeric Columns: Mean, Max, Min, Sum
    num_cols = bureau.select_dtypes(include=['number']).columns
    # Remove ID from aggregation list
    num_cols = [c for c in num_cols if c != 'SK_ID_CURR']
    
    agg_dict = {col: ['mean', 'max', 'min', 'sum'] for col in num_cols}
    
    # For Categorical Columns (Credit Type, Currency): Count/Mean
    # We manually One-Hot Encode them to aggregate
    cat_cols = bureau.select_dtypes(include=['object']).columns
    bureau_cats = pd.get_dummies(bureau[cat_cols], prefix='BUR')
    
    # Attach cats back for grouping
    bureau = pd.concat([bureau, bureau_cats], axis=1)
    
    # Add categorical aggregations (Mean = % of loans of this type)
    for cat in bureau_cats.columns:
        agg_dict[cat] = ['mean']
        
    # 3. AGGREGATE BY USER
    logger.info("Aggregating Bureau by User ID...")
    bureau_agg = bureau.groupby('SK_ID_CURR').agg(agg_dict)
    
    # Flatten Columns
    bureau_agg.columns = [f"BUREAU_{c[0]}_{c[1].upper()}" for c in bureau_agg.columns]
    
    # 4. ACTIVE LOANS SPECIFIC
    # It is very useful to know stats for ONLY "Active" loans vs "Closed" loans
    # (A closed debt from 2010 doesn't matter as much as an active one now)
    # ... For simplicity in this block, we'll stick to the global aggregate first.
    
    logger.info(f"Bureau Aggregated. Shape: {bureau_agg.shape}")
    return bureau_agg

# Execute
bureau_agg = aggregate_bureau(bb_agg)
# Clear memory (bb_agg is merged now)
del bb_agg
gc.collect()

2026-01-20 09:38:23,568 - INFO - Loading Bureau Data...
2026-01-20 09:38:28,704 - INFO - Aggregating Bureau by User ID...
2026-01-20 09:38:33,893 - INFO - Bureau Aggregated. Shape: (305811, 139)


0

In [4]:
def join_and_save(bureau_agg):
    logger.info(f"Loading Main Data from {Config.MAIN_DATA_PATH}...")
    df_main = pd.read_parquet(Config.MAIN_DATA_PATH)
    
    original_shape = df_main.shape
    
    # Left Join (Keep all Training Users, add Bureau info where available)
    # Users with NO bureau history will get NaNs (which LightGBM handles perfectly)
    df_merged = df_main.merge(bureau_agg, on='SK_ID_CURR', how='left')
    
    logger.info(f"Merged Shape: {df_merged.shape}")
    logger.info(f"Added {df_merged.shape[1] - original_shape[1]} new features from Bureau.")
    
    logger.info(f"Saving to {Config.OUTPUT_PATH}...")
    df_merged.to_parquet(Config.OUTPUT_PATH, index=False)
    
    return df_merged

# Execute
df_final = join_and_save(bureau_agg)

2026-01-20 09:39:25,778 - INFO - Loading Main Data from engineered_data.parquet...
2026-01-20 09:39:26,772 - INFO - Merged Shape: (307506, 270)
2026-01-20 09:39:26,773 - INFO - Added 139 new features from Bureau.
2026-01-20 09:39:26,774 - INFO - Saving to train_bureau_merged.parquet...



--- PREVIOUS APPS ---
Key Columns: ['SK_ID_PREV', 'SK_ID_CURR']


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0



--- INSTALLMENTS ---
Key Columns: ['SK_ID_PREV', 'SK_ID_CURR']


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000



--- POS CASH ---
Key Columns: ['SK_ID_PREV', 'SK_ID_CURR']


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0



--- CREDIT CARD ---
Key Columns: ['SK_ID_PREV', 'SK_ID_CURR']


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
